In [ ]:
!pip install mediapipe

In [ ]:
import cv2
import mediapipe as mp
from google.colab.patches import cv2_imshow
import math

In [ ]:
image = cv2.imread('img_1.jpg')
cv2_imshow(image)

#Face Detection

In [ ]:
# 初始化面部檢測模型
mp_face_detection = mp.solutions.face_detection
face_detection = mp_face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

## 官方範例程式

In [ ]:
image = cv2.imread('img_1.jpg')
# 將BGR圖像轉換為RGB，並使用MediaPipe面部檢測進行處理
results = face_detection.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
if results.detections:
  for detection in results.detections:
    
    # 印出檢測結果
    print(detection)

    # 使用官方API函式繪製檢測結果
    mp_drawing.draw_detection(image, detection)

cv2_imshow(image)

## 眼睛黑線遮蔽

In [ ]:
image = cv2.imread('img_1.jpg')
results = face_detection.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
if results.detections:
  for detection in results.detections:
    
    # 方法1: 使用官方API函式獲取標點位置
    left_eye_tip = mp_face_detection.get_key_point(detection, mp_face_detection.FaceKeyPoint.LEFT_EYE)
    right_eye_tip = mp_face_detection.get_key_point(detection, mp_face_detection.FaceKeyPoint.RIGHT_EYE)
    
    # 使用2: 使用物件導向的方式獲取標點位置
    # left_eye_tip = detection.location_data.relative_keypoints[0]
    # right_eye_tip = detection.location_data.relative_keypoints[1]
    
    # 計算標點的像素座標
    left_eye_pos = (int(image.shape[1] * left_eye_tip.x), int(image.shape[0] * left_eye_tip.y))
    right_eye_pos = (int(image.shape[1] * right_eye_tip.x), int(image.shape[0] * right_eye_tip.y))

    # 畫線
    cv2.line(image, left_eye_pos, right_eye_pos, (0,0,0), 30)

cv2_imshow(image)

## 臉部馬賽克效果

In [ ]:
# 馬賽克函式
def mosaic_effect(img, bbox, size=10):
  x, y, w, h = bbox
  new_img = img.copy()
  for m in range(y, y+h):
    for n in range(x, x+w):
      if m % size == 0 and n % size == 0:
        for i in range(0, size):
          for j in range(0, size):
            (b, g, r) = new_img[m, n]
            new_img[i + m, j + n] = (b, g, r)
  return new_img

In [ ]:
image = cv2.imread('img_1.jpg')
results = face_detection.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
if results.detections:
  for detection in results.detections:
    bbox_tip = detection.location_data.relative_bounding_box
    bbox_pos = [
      int(image.shape[1] * bbox_tip.xmin),
      int(image.shape[0] * bbox_tip.ymin),
      int(image.shape[1] * bbox_tip.width),
      int(image.shape[0] * bbox_tip.height)
    ]
    image = mosaic_effect(image, bbox_pos)
cv2_imshow(image)

# Face Mesh

In [ ]:
image = cv2.imread('img_2.jpg')
cv2_imshow(image)

In [ ]:
# 初始化面部標記模型
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
  static_image_mode=True,
  max_num_faces=1,
  refine_landmarks=True,
  min_detection_confidence=0.5
)

## 官方範例程式

In [ ]:
image = cv2.imread('img_2.jpg')
results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
if results.multi_face_landmarks:
  for face_landmarks in results.multi_face_landmarks:

    # 印出檢測結果
    print(face_landmarks)

    # 使用官方API函式繪製檢測結果
    mp_drawing.draw_landmarks(
        image=image,
        landmark_list=face_landmarks,
        connections=mp_face_mesh.FACEMESH_TESSELATION,
        landmark_drawing_spec=None,
        connection_drawing_spec=mp_drawing_styles
        .get_default_face_mesh_tesselation_style())
    mp_drawing.draw_landmarks(
        image=image,
        landmark_list=face_landmarks,
        connections=mp_face_mesh.FACEMESH_CONTOURS,
        landmark_drawing_spec=None,
        connection_drawing_spec=mp_drawing_styles
        .get_default_face_mesh_contours_style())
    mp_drawing.draw_landmarks(
        image=image,
        landmark_list=face_landmarks,
        connections=mp_face_mesh.FACEMESH_IRISES,
        landmark_drawing_spec=None,
        connection_drawing_spec=mp_drawing_styles
        .get_default_face_mesh_iris_connections_style())

cv2_imshow(image)

## 打哈欠偵測

In [ ]:
# 計算兩點距離函式
def calc_distance(point_0, point_1):
  distance = math.pow((point_0[0] - point_1[0]), 2) + math.pow((point_0[1] - point_1[1]), 2)
  distance = math.sqrt(distance)
  return distance

In [ ]:
image = cv2.imread('img_2.jpg')
results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
if results.multi_face_landmarks:
  for face_landmarks in results.multi_face_landmarks:
    point = [
      (int(image.shape[1] * face_landmarks.landmark[61].x), int(image.shape[0] * face_landmarks.landmark[61].y)),
      (int(image.shape[1] * face_landmarks.landmark[81].x), int(image.shape[0] * face_landmarks.landmark[81].y)),
      (int(image.shape[1] * face_landmarks.landmark[311].x), int(image.shape[0] * face_landmarks.landmark[311].y)),
      (int(image.shape[1] * face_landmarks.landmark[291].x), int(image.shape[0] * face_landmarks.landmark[291].y)),
      (int(image.shape[1] * face_landmarks.landmark[402].x), int(image.shape[0] * face_landmarks.landmark[402].y)),
      (int(image.shape[1] * face_landmarks.landmark[178].x), int(image.shape[0] * face_landmarks.landmark[178].y)),
    ]
    for p in point:
      cv2.circle(image, p, 2, (0, 0, 255), -1)
    ear = (calc_distance(point[1], point[5]) + calc_distance(point[2], point[4])) / (2 * calc_distance(point[0], point[3]))
    cv2.putText(image, f'ear={ear:4.2f}', point[0], cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
    
    # 判斷打哈欠則執行動作
    if ear > 0.5:
      pass

cv2_imshow(image)

## 標記深度顏色顯示

In [ ]:
image = cv2.imread('img_2.jpg')
results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
if results.multi_face_landmarks:
  for face_landmarks in results.multi_face_landmarks:
    for face_landmark in face_landmarks.landmark:
      pos = (int(image.shape[1] * face_landmark.x), int(image.shape[0] * face_landmark.y))
      depth = face_landmark.z
      if depth >= 0:
        cv2.circle(image, pos, 2, (0, 0, int(255 * depth * 50)), -1)
      else:
        cv2.circle(image, pos, 2, (int(255 * depth * -1 * 50), 0, 0), -1)

cv2_imshow(image)

## 人頭三軸選轉角度

In [ ]:
image = cv2.imread('img_2.jpg')
results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
if results.multi_face_landmarks:
  for face_landmarks in results.multi_face_landmarks:

    # 整理計算需用到的標記座標
    left_eye_left_corner = (int(image.shape[1] * face_landmarks.landmark[226].x), int(image.shape[0] * face_landmarks.landmark[226].y))
    right_eye_right_corner = (int(image.shape[1] * face_landmarks.landmark[446].x), int(image.shape[0] * face_landmarks.landmark[446].y))
    left_eye_left_corner_z = face_landmarks.landmark[226].z
    right_eye_right_corner_z = face_landmarks.landmark[446].z
    face_top_z = face_landmarks.landmark[10].z
    between_mouth_nose_z = face_landmarks.landmark[164].z

    # 計算3軸選轉角度
    roll = math.atan((right_eye_right_corner[1] - left_eye_left_corner[1]) / (right_eye_right_corner[0] - left_eye_left_corner[0])) * 180 / math.pi
    yaw = (right_eye_right_corner_z - left_eye_left_corner_z) * 100
    pitch = (face_top_z - between_mouth_nose_z) * 100
    
    print(f'roll = {roll}')
    print(f'yaw = {yaw}')
    print(f'pitch = {pitch}')

    print_pos = [226, 446, 10, 164]
    for p in print_pos:
      face_landmark = face_landmarks.landmark[p]
      pos = (int(image.shape[1] * face_landmark.x), int(image.shape[0] * face_landmark.y))
      cv2.circle(image, pos, 4, (0, 0, 255), -1)

cv2_imshow(image)

# Hands

In [ ]:
image = cv2.imread('img_3.jpg')
cv2_imshow(image)

In [ ]:
# 初始化手勢檢測模型
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
  static_image_mode=True,
  max_num_hands=4,
  min_detection_confidence=0.1
)

## 官方範例程式

In [ ]:
image = cv2.imread('img_3.jpg')
results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
print('Handedness:', results.multi_handedness)
if results.multi_hand_landmarks:
  image_height, image_width, _ = image.shape
  for hand_landmarks in results.multi_hand_landmarks:
    print('hand_landmarks:', hand_landmarks)
    print(
        f'Index finger tip coordinates: (',
        f'{hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x * image_width}, '
        f'{hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y * image_height})'
    )
    mp_drawing.draw_landmarks(
        image,
        hand_landmarks,
        mp_hands.HAND_CONNECTIONS,
        mp_drawing_styles.get_default_hand_landmarks_style(),
        mp_drawing_styles.get_default_hand_connections_style())
  cv2_imshow(cv2.flip(image, 1))
  # Draw hand world landmarks.
  if results.multi_hand_world_landmarks:
    for hand_world_landmarks in results.multi_hand_world_landmarks:
      mp_drawing.plot_landmarks(
        hand_world_landmarks, mp_hands.HAND_CONNECTIONS, azimuth=5)

## 手勢判斷

In [ ]:
def vector_2d_angle(v1,v2): # 求出v1,v2兩條向量的夾角
    v1_x=v1[0]
    v1_y=v1[1]
    v2_x=v2[0]
    v2_y=v2[1]
    try:
        angle_= math.degrees(math.acos((v1_x*v2_x+v1_y*v2_y)/(((v1_x**2+v1_y**2)**0.5)*((v2_x**2+v2_y**2)**0.5))))
    except:
        angle_ = 100000.
    return angle_

def hand_angle(hand_):
    angle_list = []
    #---------------------------- thumb 大拇指角度
    angle_ = vector_2d_angle(
        ((int(hand_[0][0])- int(hand_[2][0])),(int(hand_[0][1])-int(hand_[2][1]))),
        ((int(hand_[3][0])- int(hand_[4][0])),(int(hand_[3][1])- int(hand_[4][1])))
        )
    angle_list.append(angle_)
    #---------------------------- index 食指角度
    angle_ = vector_2d_angle(
        ((int(hand_[0][0])-int(hand_[6][0])),(int(hand_[0][1])- int(hand_[6][1]))),
        ((int(hand_[7][0])- int(hand_[8][0])),(int(hand_[7][1])- int(hand_[8][1])))
        )
    angle_list.append(angle_)
    #---------------------------- middle 中指角度
    angle_ = vector_2d_angle(
        ((int(hand_[0][0])- int(hand_[10][0])),(int(hand_[0][1])- int(hand_[10][1]))),
        ((int(hand_[11][0])- int(hand_[12][0])),(int(hand_[11][1])- int(hand_[12][1])))
        )
    angle_list.append(angle_)
    #---------------------------- ring 無名指角度
    angle_ = vector_2d_angle(
        ((int(hand_[0][0])- int(hand_[14][0])),(int(hand_[0][1])- int(hand_[14][1]))),
        ((int(hand_[15][0])- int(hand_[16][0])),(int(hand_[15][1])- int(hand_[16][1])))
        )
    angle_list.append(angle_)
    #---------------------------- pink 小拇指角度
    angle_ = vector_2d_angle(
        ((int(hand_[0][0])- int(hand_[18][0])),(int(hand_[0][1])- int(hand_[18][1]))),
        ((int(hand_[19][0])- int(hand_[20][0])),(int(hand_[19][1])- int(hand_[20][1])))
        )
    angle_list.append(angle_)
    return angle_list

def get_hand_angle(image, hand_landmarks):
  keypoint_pos = []
  for i in range(21):
    x = hand_landmarks.landmark[i].x * image.shape[1]
    y = hand_landmarks.landmark[i].y * image.shape[0]
    keypoint_pos.append((x,y))
  angle_list = hand_angle(keypoint_pos)
  return angle_list


In [ ]:
image = cv2.imread('img_3.jpg')
results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

if results.multi_hand_landmarks:
  for hand_landmarks in results.multi_hand_landmarks:
    
    angle_list = get_hand_angle(image, hand_landmarks)
    print(angle_list)

    mp_drawing.draw_landmarks(
      image,
      hand_landmarks,
      mp_hands.HAND_CONNECTIONS,
      mp_drawing_styles.get_default_hand_landmarks_style(),
      mp_drawing_styles.get_default_hand_connections_style())
    
    hand_pos = (int(image.shape[1] * hand_landmarks.landmark[0].x), int(image.shape[0] * hand_landmarks.landmark[0].y))
    if angle_list[0] > 50 and angle_list[4] > 50 and angle_list[1] < 30 and angle_list[2] < 30 and angle_list[3] < 30:
      cv2.putText(image, '3', hand_pos, cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
    if angle_list[0] > 50 and angle_list[1] > 50 and angle_list[2] < 30 and angle_list[3] < 30 and angle_list[4] < 30:
      cv2.putText(image, 'OK', hand_pos, cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

cv2_imshow(image)

# Pose (自行練習)